# Applied Data Science Capstone

## Assignment 2: Segmenting and Clustering Neighborhoods in Toronto - Part  3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
 - To add enough Markdown cells to explain what you decided to do and to report any observations you make.
 - To generate maps to visualize you neighborhoods and how they cluster together.

Once you are happy with your analysis, submit a link to a new Notebook on your Github repository. (3 marks)

### Import the necessary libraries

In [1]:
# for handling data in a vectorized manner
import numpy as np  

# for data analysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# to handle json files
import json  

# to convert address to latitude and longitude
from geopy.geocoders import Nominatim

# to handle requests
import requests

# to transform JSON file to pandas DataFrame
from pandas.io.json import json_normalize

# For data visualization
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means for clustering
from sklearn.cluster import KMeans

# map rendering library
import folium

print('Libraries are imported! You are Good to Go --> ')

Libraries are imported! You are Good to Go --> 


In [2]:
df = pd.read_csv('final_data.csv', index_col = 0)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### To count the number of Boroughs and Neighborhoods

In [3]:
boroughs_count = len(df['Borough'].unique())
neighborhoods_count = df.shape[0]
print(f'The dataframe has {boroughs_count} and {neighborhoods_count} neighborhoods!')

The dataframe has 10 and 103 neighborhoods!


### Use geopy library to get the latitude and longitude values of Toronto, Canada

In [4]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geographical coordinate of Toronto, Canada is {latitude}, {longitude}')

The geographical coordinate of Toronto, Canada is 43.6534817, -79.3839347


### Create a map of Toronto, Canada with neighborhoods superimposed on top

In [5]:
# create map of Toronto, Canada using latitude and longitude values

map_toronto = folium.Map(locations = [latitude, longitude], zoom_start = 10)

# add markers to map

for lat, long, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = f'{neighborhood, borough}'
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, long], radius = 5, popup = label, color = 'blue', fill = True, fill_color = '#3186cc', fill_opacity = 0.7,
                       parse_html = False).add_to(map_toronto)

map_toronto

In [6]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
scarborough_data = df.query('Borough == "Scarborough"')
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Let's get the Geographical coordinates of Scarborough

In [8]:
address = 'Scarborough, Ontario'

geolocator = Nominatim(user_agent = 'scarborough_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geographical coordinate of Scarborough, Ontario, Toronto is {latitude}, {longitude}')

The geographical coordinate of Scarborough, Ontario, Toronto is 43.773077, -79.257774


In [9]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, long, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, long], popup = label, color = 'blue', fill = True, fill_color = '#3186cc', fill_opacity = 0.7, parse_html = False).add_to(map_scarborough)

map_scarborough

### Utilizing the Foursquare API to explore the neighborhoods and segment them

In [10]:
CLIENT_ID = 'FCOTPNVLCVK0EFB5WQEW3TE51Y5AG1MHARD2W0OQ4Q2VG4WG'
CLIENT_SECRET = 'KPIH3QP3A4OEDQ05DBH41XG2NNNMTVTBUDRWAFONAQ2H3SEI'
VERSION = '20180605'

print('Credentials:')
print(f'CLIENT_ID: {CLIENT_ID}')
print(f'CLIENT_SECRET: {CLIENT_SECRET}')

Credentials:
CLIENT_ID: FCOTPNVLCVK0EFB5WQEW3TE51Y5AG1MHARD2W0OQ4Q2VG4WG
CLIENT_SECRET: KPIH3QP3A4OEDQ05DBH41XG2NNNMTVTBUDRWAFONAQ2H3SEI


### Exploring the First Neighborhood in the Dataframe

In [11]:
scarborough_data.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [12]:
neighborhood_lat = scarborough_data.loc[0, 'Latitude']
neighborhood_long = scarborough_data.loc[0, 'Longitude']
neighborhood_name = scarborough_data.loc[0, 'Neighborhood']

print(f'Latitude and Longitude values of {neighborhood_name} are {neighborhood_lat}, {neighborhood_long}')

Latitude and Longitude values of Malvern, Rouge are 43.8066863, -79.19435340000003


### Getting the top 100 venues that are in Malvern within a radius of 500 metres 

In [51]:
LIMIT = 100
RADIUS = 500

# creating the url for the query
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={neighborhood_lat},{neighborhood_long}&radius={RADIUS}&limit={LIMIT}'

url

'https://api.foursquare.com/v2/venues/explore?&client_id=FCOTPNVLCVK0EFB5WQEW3TE51Y5AG1MHARD2W0OQ4Q2VG4WG&client_secret=KPIH3QP3A4OEDQ05DBH41XG2NNNMTVTBUDRWAFONAQ2H3SEI&v=20180605&ll=43.8066863,-79.19435340000003&radius=500&limit=100'

In [52]:
# making a GET request
results = requests.get(url).json()['response']['groups'][0]['items']
results

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4bb6b9446edc76b0d771311c',
   'name': 'Wendy’s',
   'location': {'crossStreet': 'Morningside & Sheppard',
    'lat': 43.80744841934756,
    'lng': -79.19905558052072,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.80744841934756,
      'lng': -79.19905558052072}],
    'distance': 387,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d16e941735',
     'name': 'Fast Food Restaurant',
     'pluralName': 'Fast Food Restaurants',
     'shortName': 'Fast Food',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4bb6b9446edc76b0d771311c-0'},
 {'reasons':

In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [64]:
print(scarborough_venues.shape)
scarborough_venues.head()

(95, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


### Let's find out the number of unique Venue Categories

In [65]:
print(f'The number of unique categories are {len(scarborough_venues["Venue Category"].unique())}')

The number of unique categories are 55


In [66]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood coulmn back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood']

# move neighborhood column to the first column
fixed_column = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_column]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
# shape of the new dataframe
scarborough_onehot.shape

(95, 56)

In [69]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.0,0.250000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.25,0.25,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.250000,0.00,0.0,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.2,0.000,0.000000,0.000000,0.2,0.0,0.000000,0.000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.2,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.200000,0.00,0.0,0.000000,0.0
2,Cedarbrae,0.000000,0.125,0.125,0.125000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.125,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.125000,0.125000,0.0,0.000000,0.125,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.125000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.000,0.071429,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.071429,0.071429,0.0,0.0,0.071429,0.000,0.000000,0.000000,0.0,0.071429,0.071429,0.071429,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.071429,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.071429,0.0,0.0,0.071429,0.142857,0.0,0.0,0.000,0.000000,0.071429,0.000000,0.00,0.0,0.071429,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.333333,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.333333,0.00,0.0,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.200000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.40,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.2,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.200,0.000000,0.0,0.000000,0.000000,0.2,0.100,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.1,0.00,0.100,0.000000,0.00,0.00,0.00,0.000,0.1,0.000,0.00,0.000000,0.000,0.000000,0.1,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.000,0.000,0.125000,0.0,0.125000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.125,0.000000,0.00,0.00,0.00,0.125,0.0,0.125,0.00,0.000000,0.125,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.125,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0
8,"Kennedy Park, Ionview

In [70]:
scarborough_grouped.shape

(16, 56)

### Let's print each neighborhood along with the top 5 most common venues

In [72]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq' : 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink   0.2
1  General Entertainment   0.2
2                   Café   0.2
3                   Farm   0.2
4        College Stadium   0.2


----Cedarbrae----
                 venue  freq
0               Bakery  0.12
1                 Bank  0.12
2      Thai Restaurant  0.12
3  Fried Chicken Joint  0.12
4   Athletics & Sports  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                 venue  freq
0          Pizza Place  0.14
1    Convenience Store  0.07
2   Chinese Restaurant  0.07
3          Gas Station  0.07
4  Fried Chicken Joint  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1         Skat

In [73]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


## Clustering Neighborhoods

In [86]:
# set number of clusters
kclusters = 5
scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(scarborough_grouped_clustering)

#check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

numpy.ndarray

In [105]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#scarborough_merged.drop(scarborough_merged.index[16], inplace = True)
scarborough_merged['Cluster Labels'] = scarborough_merged['Cluster Labels'].astype(int)

# scarborough_merged.head() # check the last columns!

In [106]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [108]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store


### Cluster 2

In [109]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,1,American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
12,Scarborough,1,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


### Cluster 3

In [110]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


### Cluster 4

In [111]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,3,Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
3,Scarborough,3,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
4,Scarborough,3,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
6,Scarborough,3,Discount Store,Coffee Shop,Bus Station,Department Store,Chinese Restaurant,Hobby Shop,Convenience Store,Construction & Landscaping,Grocery Store,General Entertainment
7,Scarborough,3,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Intersection,Park,Bus Station,Electronics Store,Department Store
9,Scarborough,3,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,3,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
11,Scarborough,3,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,3,Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
15,Scarborough,3,Fast Food Restaurant,Chinese Restaurant,Bubble Tea Shop,Coffee Shop,Discount Store,Electronics Store,Pharmacy,Pizza Place,Grocery Store,Breakfast Spot


### Cluster 5

In [112]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4,Playground,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
14,Scarborough,4,Playground,Park,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


# Conclusions

## 1. Cluster 4 is the biggest.
## 2. Clusters 2 and 5 have two neighborhoods each.
## 3. Clusters 1 and 3 have only one neighborhood. 